In [1]:
import time
import re
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:

CHROMEDRIVER_PATH = "C:/Users/nandhu S/OneDrive/Desktop/eventradar_project/chromedriver.exe"

service = Service(CHROMEDRIVER_PATH)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
driver = webdriver.Chrome(service=service, options=options)


In [ ]:

url = "https://devpost.com/hackathons?status[]=upcoming"
driver.get(url)
time.sleep(5)  
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.find_all("a", class_="tile-anchor")
events = []

for card in cards:
    try:
        title = card.find("h3").text.strip()
        link = card['href']
        full_url = "https://devpost.com" + link if link.startswith("/") else link
        location = card.find("div", class_="info").text.strip()
        
        date_range = card.find("div", class_="submission-period")
        event_date = date_range.text.strip() if date_range else ""
        host = card.find("span", class_="host-label")
        speakers_orgs = host.get("title") if host else ""
        tags = [tag.text.strip() for tag in card.find_all("span", class_="theme-label")]
        events.append({
            "event_title": title,
            "platform": "Devpost",
            "event_type": "Hackathon",
            "event_date": event_date,
            "location": location,
            "speakers_orgs": speakers_orgs,
            "topic_tags": ", ".join(tags),
            "url": full_url,
            "audience_segment": ""  
        })
    except Exception as e:
        print("⚠️ Error parsing event:", e)

df_raw = pd.DataFrame(events)
df_raw.to_csv("step1_raw_devpost_events.csv", index=False)
df_raw.head()


,event_title,platform,event_type,event_date,location,speakers_orgs,topic_tags,url,audience_segment
0,LaunchHacks IV,Devpost,Hackathon,"Jun 28 - Jul 01, 2025",Online,LaunchHacks,"Beginner Friendly, Education, Open Ended",https://launchhacks-iv.devpost.com/?ref_featur...,
1,United Hacks V5,Devpost,Hackathon,"Jul 11 - 13, 2025",Online,Hack United,"Beginner Friendly, Open Ended, Social Good",https://unitedhacksv5.devpost.com/?ref_feature...,
2,NextStep Hacks 2025,Devpost,Hackathon,"Jul 11 - 14, 2025",Online,HackAlphaX,"Beginner Friendly, Social Good, Machine Learni...",https://nextstep2025.devpost.com/?ref_feature=...,
3,Proof of Concept,Devpost,Hackathon,"Jun 26 - Aug 21, 2025",Online,XION,"Beginner Friendly, Machine Learning/AI, Mobile",https://proofofconcept.devpost.com/?ref_featur...,
4,HackVortex Codestorm 5,Devpost,Hackathon,"Aug 08 - 17, 2025",Online,HackVortex,"Beginner Friendly, Health, Machine Learning/AI",https://hackvortex-codestorm-5.devpost.com/?re...,
